In [ ]:
import os
os.chdir("..")

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np
from numpy.lib import recfunctions
from matplotlib.patches import Rectangle
import h5py
from scipy.optimize import curve_fit
from scipy.integrate import quad

In [ ]:
datafolder = "data/"
imagefolder = "figures/"

In [ ]:
pd.read_hdf(datafolder+"midranged.h5", key="galaxies", start=5, stop=10)

In [ ]:
df = pd.read_hdf(datafolder+"midranged.h5", key="galaxies", columns=["r3d","rproj","vproj","Vradnorm","hostrow"])
df

In [ ]:
df = df.loc[df["r3d"] > 2]
df.index = pd.RangeIndex(df.shape[0])
df

In [ ]:
def correction(R, A, alpha):
	return 1/ 10 * R - A * R**alpha

In [ ]:
result = curve_fit(correction, df["r3d"], df["Vradnorm"], p0=[1, -0.5], bounds=([0,-3],[1,0]))

In [ ]:
result

In [ ]:
df["Vradcorr"] = correction(df["r3d"], *result[0])
df["Vcentered"] = df["Vradnorm"] - df["Vradcorr"]

In [ ]:
df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
sb.histplot(data=df, x="r3d", y="Vradnorm", bins=100, ax=ax)
xs = np.linspace(2,30, 1000)
ax.plot(xs, correction(xs, *result[0]), color="red", label="Correction")
ax.plot(xs, 1/10 * xs, color="black", linestyle="--", label="Hubble Flow")
ax.legend()

In [ ]:
sb.histplot(data=df, x="r3d", y="Vcentered", bins=100)

In [ ]:
def gaussian(x, sigma):
	return 1 / np.sqrt(2*np.pi*sigma**2) * np.exp(-0.5*(x/sigma)**2)

In [ ]:
hist, bins = np.histogram(df["Vcentered"], bins=100, density=True)

In [ ]:
hist.shape, bins.shape

In [ ]:
secondresult = curve_fit(gaussian, (bins[1:]+bins[:-1])/2, hist, p0=[0.6], bounds=(0,1))

In [ ]:
plt.bar(bins[:-1], hist, width=np.diff(bins), align='edge', alpha=0.5, label="Data")
xs = np.linspace(bins[0], bins[-1], 1000)
plt.plot(xs, gaussian(xs, *secondresult[0]), label="Gaussian fit", color="red")
plt.xlabel("Vcentered")
plt.ylabel("Counts")
plt.legend()

In [ ]:
result, secondresult

In [ ]:
Rhist, Rbins = np.histogram(df["r3d"], bins=100, density=True)

In [ ]:
H0 = lambda x: 1 / 10
PVcorr = lambda x: correction(x, *result[0])
PU = lambda x: gaussian(x, *secondresult[0])
PR = lambda x: np.interp(x, (Rbins[1:]+Rbins[:-1])/2, Rhist)
def unnormalizedPosterior(R, r, v):
	if np.isscalar(R):
		if R < r:
			return 0
		else:
			return r/R/np.sqrt(R**2-r**2) * PR(R) * PU(np.abs(v)-(PVcorr(R))*(np.cos(np.arcsin(r/R))))
	value = np.zeros_like(R)
	mask = R >= r
	value[mask] = r/R[mask]/np.sqrt(R[mask]**2-r**2) * PR(R[mask]) * PU(np.abs(v)-(PVcorr(R[mask]))*(np.cos(np.arcsin(r/R[mask]))))
	return value
def normalizedPosterior(R, r, v):
	if np.isscalar(R):
		if R < r:
			return 0
		unnormalized_value = unnormalizedPosterior(R, r, v)
		normalization = quad(unnormalizedPosterior, r, 30, args=(r, v))[0]
		if normalization == 0:
			return 0
		return unnormalized_value / normalization
	value = unnormalizedPosterior(R, r, v)
	normalization = quad(unnormalizedPosterior, r, 30, args=(r, v))[0]
	if normalization == 0:
		return 0
	return value / normalization
def firstComponent(R, r, v):
	value = np.zeros_like(R)
	mask = R >= r
	value[mask] = r/R[mask]/np.sqrt(R[mask]**2-r**2)
	return value
def secondComponent(R, r, v):
	value = np.zeros_like(R)
	mask = R >= r
	value[mask] = PU(np.abs(v)-(PVcorr(R[mask]))*(np.cos(np.arcsin(r/R[mask]))))
	return value

In [ ]:
xs = np.linspace(0,20,1001)
plt.plot(xs, [secondComponent(x, 1, 0) for x in xs], label="1,0")
plt.plot(xs, [secondComponent(x, 1, 1) for x in xs], label="1,1")
plt.plot(xs, [secondComponent(x, 1, 2) for x in xs], label="1,2")
plt.plot(xs, [secondComponent(x, 1, 3) for x in xs], label="1,3")
plt.plot(xs, [secondComponent(x, 5, 1) for x in xs], label="5,1")
plt.plot(xs, [secondComponent(x, 5, 2) for x in xs], label="5,2")
plt.plot(xs, [secondComponent(x, 5, 3) for x in xs], label="5,3")
plt.legend()

In [ ]:
plt.hist(df["r3d"], bins=100, density=True, alpha=0.5, label="Data")
xs = np.linspace(df["r3d"].min(), df["r3d"].max(), 1000)
plt.plot(xs, PR(xs), label="R prior", color="red")
plt.xlabel("R")
plt.ylabel("Counts")
plt.legend()

In [ ]:
ntrials = 15
elected = np.random.choice(df.index, ntrials, replace=False)

In [ ]:
fig, axs = plt.subplots(ntrials, 1, figsize=(10, 2*ntrials), sharex=True)
for i, idx in enumerate(elected):
	r = df["rproj"].loc[idx]
	v = df["vproj"].loc[idx]
	Rtrue = df["r3d"].loc[idx]
	R = np.linspace(0, 30, 1000)
	posterior = [unnormalizedPosterior(R3d, r, v) for R3d in R]
	
	axs[i].plot(R, posterior, label=f"r={r:.2f}, v={v:.2f}")
	axs[i].axvline(Rtrue, color="red", linestyle="--", label=f"Rtrue={Rtrue:.2f}")
	axs[i].set_xlabel("R")
	axs[i].set_ylabel("Posterior")
	axs[i].legend()
fig.tight_layout()
fig.savefig(imagefolder + f"posterior.png", dpi=300)

In [ ]:
fig, axs = plt.subplots(ntrials, 2, figsize=(10, 2*ntrials), sharex=True)
for i, idx in enumerate(elected):
	r = df["rproj"].loc[idx]
	v = df["vproj"].loc[idx]
	Rtrue = df["r3d"].loc[idx]
	R = np.linspace(0, 30, 1000)
	posterior1 = [firstComponent(R3d, r, v) for R3d in R]
	posterior2 = [secondComponent(R3d, r, v) for R3d in R]
	
	axs[i,0].plot(R, posterior1, label=f"r={r:.2f}, v={v:.2f}")
	axs[i,1].plot(R, posterior2, label=f"r={r:.2f}, v={v:.2f}")
	axs[i,0].axvline(Rtrue, color="red", linestyle="--", label=f"Rtrue={Rtrue:.2f}")
	axs[i,1].axvline(Rtrue, color="red", linestyle="--", label=f"Rtrue={Rtrue:.2f}")
	axs[i,0].set_xlabel("R")
	axs[i,0].set_ylabel("Posterior")
	axs[i,0].legend()
axs[0,0].set_title("Radial contribution")
axs[0,1].set_title("Velocity contribution")
fig.tight_layout()
fig.savefig(imagefolder + f"posterior_components.png", dpi=300)

In [ ]:
rbins = 15
vbins = 15
df["rproj_bin"] = pd.cut(df["rproj"], bins=rbins, labels=False)
df["vproj_bin"] = pd.cut(df["vproj"], bins=vbins, labels=False)

In [ ]:
df

In [ ]:
df["rproj_bin"].unique()

In [ ]:
df_test = df.sample(frac=0.001)

In [ ]:
fig, axs = plt.subplots(rbins, vbins, figsize=(3*rbins, 3*vbins))

for (rbin, vbin), group in df_test.groupby(["rproj_bin", "vproj_bin"]):
	ax = axs[rbin, vbin]
	ax.legend(title=f"rproj bin {rbin}, vproj bin {vbin}")
	
	if group.shape[0] == 0:
		continue
	
	Rs = np.linspace(0, 30, 1000)
	
	posteriors = np.array([normalizedPosterior(Rs, r, v) for r,v in group[["rproj", "vproj"]].values])
	posterior = posteriors.mean(axis=0)

	ax.plot(Rs, posterior, label="Posterior")
	ax.hist(group["r3d"], bins=50, density=True, alpha=0.5, label="Data", color="gray")
	
fig.tight_layout()
fig.savefig(imagefolder + f"fullcheck.png")
fig.savefig(imagefolder + f"fullcheck.pdf")